In [ ]:
##Imports 
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import pandas as pd
import numpy as np

In [ ]:
"""Main code for bike data
    scrapping till the specs step""" 
# Note - might raise exceptions when a popup appears in between the scappring which leads to a stopage on a point, due to this navigation approach 
# Navigate back to the main bike page 
# driver.execute_script("window.history.go(-2)")" as "-2" defines the procedure to go 2 pages back in history point.
# We can bypass it by simply closing the pop-up then manually moving towards the main bikelist page it will retry automatically. 

#perfect one ,"ran successfully for the whole list "

def find_element_with_retry(driver, by, value, retries=3, delay=2):
    for _ in range(retries):
        try:
            element = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except (TimeoutException, StaleElementReferenceException) as e:
            print(f"Error finding element: {str(e)}. Retrying...")
            sleep(delay)
    raise Exception(f"Unable to find element {value} after {retries} retries.")

def select_dropdown_option(driver):
    dropdown_option_xpath = '//*[@id="rf01"]/header/div[1]/div/div/div[2]/div/div/form/div/div/div/ul/li'
    try:
        dropdown_options = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, dropdown_option_xpath))
        )
        if dropdown_options:
            dropdown_options[0].click()  # Clicking the first option
            return True
    except Exception as e:
        print(f"Error selecting dropdown option: {str(e)}")
        return False

def scrape_bike_details(search_query, driver):
    bike_details_list = []
    try:
        # Find the search text input element
        search_text_input = find_element_with_retry(driver, By.ID, 'cardekhosearchtext')

        # Input the search query into the search text input
        search_text_input.clear()
        search_text_input.send_keys(search_query)

        # Wait for a while to let the dropdown options appear
        sleep(1)

        # Re-find the dropdown options after waiting
        if not select_dropdown_option(driver):
            print(f"Failed to select dropdown option for query: {search_query}")
            return bike_details_list

        sleep(1)

        bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')

        for index in range(len(bike_list_element.find_elements(By.TAG_NAME, 'li'))):
            try:
                # Re-find the bike list element to avoid stale element reference
                bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')
                
                item = bike_list_element.find_elements(By.TAG_NAME, 'li')[index]
                x_element = item.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a')
                link = x_element.get_attribute('href')
                
                # Move to the element to ensure it's clickable
                ActionChains(driver).move_to_element(x_element).click().perform()

                # Additional scraping from the new page (e.g., bike name, price, etc.)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h1')))
                bike_name = driver.find_element(By.XPATH, '//h1').text

                # Fetch the price
                bike_price_element = driver.find_element(By.XPATH, '//div[@class="price"]')
                bike_price_text = bike_price_element.text.split('Rs.')[-1].split('*')[0].strip()

                # Check if the price contains a range
                if '-' in bike_price_text:
                    min_price, max_price = bike_price_text.split('-')
                    bike_price = f"{min_price.strip()} Lakh - {max_price.strip()}"
                else:
                    bike_price = f"{bike_price_text}"

                # Click on the "Specs" button to navigate to the specifications page
                specs_button = find_element_with_retry(driver, By.XPATH, '//a[contains(text(), "Specs")]')

                # Move to the element to ensure it's clickable
                ActionChains(driver).move_to_element(specs_button).click().perform()

                # Wait for the technical specifications section to be present
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'technicalSpecs')))

                # Find the technical specifications block
                tech_specs_block = driver.find_element(By.ID, 'technicalSpecs')

                # Find all h3 elements within the technical specifications block
                section_titles = tech_specs_block.find_elements(By.TAG_NAME, 'h3')

                for title_element in section_titles:
                    section_title = title_element.text
                    section_data = {}

                    # Find the corresponding table for each section
                    table = title_element.find_element(By.XPATH, './following-sibling::table[1]')

                    # Find all rows in the table
                    rows = table.find_elements(By.TAG_NAME, 'tr')

                    for row in rows:
                        columns = row.find_elements(By.TAG_NAME, 'td')
                        if len(columns) == 2:
                            feature = columns[0].text
                            value = columns[1].text
                            section_data[feature] = value

                    # Add the section data to the bike details list
                    bike_details_list.append({
                        'Search Query': search_query,
                        'Bike Name': bike_name,
                        'Price': bike_price,
                        'Section Title': section_title,
                        'Section Data': section_data
                    })

                # Navigate back to the main bike page
                driver.execute_script("window.history.go(-2)")
                
                # Wait for the bike list to load again
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'bikelist')))
                
            except NoSuchElementException as ne:
                print(f"No <h3> tag found for item {search_query}. Skipping...")
                continue
            except Exception as e:
                print(f"An error occurred while processing item {search_query}: {str(e)}")
                continue

    except NoSuchElementException as ne:
        print(f"No element found: {ne}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    return bike_details_list

try:
    bike_details_list = []

    with webdriver.Chrome() as driver:
        driver.get('https://www.bikedekho.com')

        search_queries = ['Bajaj bike', 'TVS bike', 'Hero bike', 'Royal Enfield bike','Honda bike', 'Yamaha bike', 'Suzuki bike','KTM bike',
            'Jawa bike', 'Benelli bike', 'Harley bike', 'Triumph bike',
            'Ducati bike', 'BMW bike', 'Kawasaki bike', 'Husqvarna bike',
            'Revolt bike', 'Ather bike','Ola bike','Aprilia Bikes']
        
        for query in search_queries:
            result = scrape_bike_details(query, driver)
            bike_details_list.extend(result)

    # Convert the list of dictionaries to a pandas DataFrame
    dv = pd.DataFrame(bike_details_list)
    
    # # Save the DataFrame to a CSV file
    # dfz.to_csv('bike_details.csv', index=False)

except Exception as e:
    print(f"An error occurred: {str(e)}")


In [ ]:
# df6 = df6.reset_index(drop=True)


### working for specs page link extraction 

In [1]:
#working for link extraction 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from time import sleep
import pandas as pd

def find_element_with_retry(driver, by, value, retries=3, delay=2):
    for _ in range(retries):
        try:
            element = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except (TimeoutException, NoSuchElementException) as e:
            print(f"Error finding element: {str(e)}. Retrying...")
            sleep(delay)
    raise Exception(f"Unable to find element {value} after {retries} retries.")

def select_dropdown_option(driver):
    dropdown_option_xpath = '//*[@id="rf01"]/header/div[1]/div/div/div[2]/div/div/form/div/div/div/ul/li'
    try:
        dropdown_options = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, dropdown_option_xpath))
        )
        if dropdown_options:
            dropdown_options[0].click()  # Clicking the first option
            return True
    except Exception as e:
        print(f"Error selecting dropdown option: {str(e)}")
        return False

def scrape_bike_specs_links(search_query, driver):
    bike_links_list = []
    try:
        # Find the search text input element
        search_text_input = find_element_with_retry(driver, By.ID, 'cardekhosearchtext')

        # Input the search query into the search text input
        search_text_input.clear()
        search_text_input.send_keys(search_query)

        # Wait for a while to let the dropdown options appear
        sleep(1)

        # Re-find the dropdown options after waiting
        if not select_dropdown_option(driver):
            print(f"Failed to select dropdown option for query: {search_query}")
            return bike_links_list

        sleep(1)

        bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')

        for index in range(len(bike_list_element.find_elements(By.TAG_NAME, 'li'))):
            try:
                # Re-find the bike list element to avoid stale element reference
                bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')
                
                item = bike_list_element.find_elements(By.TAG_NAME, 'li')[index]
                x_element = item.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a')
                link = x_element.get_attribute('href')

                # Navigate to the bike's page
                driver.get(link)
                
                # Find the "Specs" link
                specs_link_element = find_element_with_retry(driver, By.XPATH, '//a[contains(@title, "Specifications")]')
                specs_link = specs_link_element.get_attribute('href')
                
                bike_links_list.append({'Search Query': search_query, 'Specs Link': specs_link})
                print(f"Scraped Specs link for {search_query}: {specs_link}")

                # Navigate back to the main bike page
                driver.execute_script("window.history.go(-1)")
                
                # Wait for the bike list to load again
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'bikelist')))
                
            except NoSuchElementException as ne:
                print(f"No <h3> tag found for item {search_query}. Skipping...")
                continue
            except Exception as e:
                print(f"An error occurred while processing item {search_query}: {str(e)}")
                continue

    except NoSuchElementException as ne:
        print(f"No element found: {ne}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    return bike_links_list

try:
    bike_links_list = []

    with webdriver.Chrome() as driver:
        driver.get('https://www.bikedekho.com')
        search_queries = ['yezdi bikes','ultraviolet bike',]
        # search_queries = ['Bajaj bike', 'TVS bike', 'Hero bike', 'Royal Enfield bike','Honda bike', 'Yamaha bike', 'Suzuki bike','KTM bike',
        #     'Jawa bike', 'Benelli bike', 'Harley bike', 'Triumph bike',
        #     'Ducati bike', 'BMW bike', 'Kawasaki bike', 'Husqvarna bike',
        #     'Revolt bike', 'Ather bike','Ola bike','Aprilia Bikes']
        
        for query in search_queries:
            result = scrape_bike_specs_links(query, driver)
            bike_links_list.extend(result)

    # Convert the list of dictionaries to a pandas DataFrame
    dfo = pd.DataFrame(bike_links_list)
    print(dfo)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")


Scraped Specs link for yezdi bikes: https://www.bikedekho.com/yezdi-motorcycles/cruiser/specifications
Scraped Specs link for yezdi bikes: https://www.bikedekho.com/yezdi-motorcycles/adventure/specifications
Scraped Specs link for yezdi bikes: https://www.bikedekho.com/yezdi-motorcycles/roadking/specifications
Scraped Specs link for ultraviolet bike: https://www.bikedekho.com/ultraviolette/f77/specifications
Scraped Specs link for ultraviolet bike: https://www.bikedekho.com/ultraviolette/f77/specifications
       Search Query                                         Specs Link
0       yezdi bikes  https://www.bikedekho.com/yezdi-motorcycles/cr...
1       yezdi bikes  https://www.bikedekho.com/yezdi-motorcycles/ad...
2       yezdi bikes  https://www.bikedekho.com/yezdi-motorcycles/ro...
3  ultraviolet bike  https://www.bikedekho.com/ultraviolette/f77/sp...
4  ultraviolet bike  https://www.bikedekho.com/ultraviolette/f77/sp...


In [2]:
dfo

,Search Query,Specs Link
0,yezdi bikes,https://www.bikedekho.com/yezdi-motorcycles/cr...
1,yezdi bikes,https://www.bikedekho.com/yezdi-motorcycles/ad...
2,yezdi bikes,https://www.bikedekho.com/yezdi-motorcycles/ro...
3,ultraviolet bike,https://www.bikedekho.com/ultraviolette/f77/sp...
4,ultraviolet bike,https://www.bikedekho.com/ultraviolette/f77/sp...


In [ ]:
# duplicates = dfi[dfi.duplicated(subset=['Search Query', 'Specs Link'], keep=False)]
# print(duplicates)

# # Convert the list of dictionaries to a pandas DataFrame
# # dfv = pd.DataFrame(bike_links_list)

# Drop duplicate rows based on 'Search Query' and 'Specs Link'
# dfi = dfi.drop_duplicates(subset=['Search Query', 'Specs Link'])

# print(dfi)

In [8]:
# # Convert the list of dictionaries to a pandas DataFrame
# # dfv = pd.DataFrame(bike_links_list)

# # Drop duplicate rows based on 'Search Query' and 'Specs Link'
dffinal = dffinal.drop_duplicates(subset=['Search Query', 'Specs Link'])

# print(df6)


In [3]:
dfo.to_csv('bike_spec_page_link_Yezdi_ultravioet.csv', index=False)
# 

In [60]:
# import pandas as pd

# # Read the CSV files into pandas DataFrames
df8 = pd.read_csv('bike_specs_yezdi_ultraviolet.csv')
df9 = pd.read_csv('bike_specall.csv')

# # # Concatenate the DataFrames
df_concatenated = pd.concat([df8, df9], ignore_index=True)

# # # # Optionally, reset the index
df_concatenated = df_concatenated.reset_index(drop=True)

# # # # Save the concatenated DataFrame to a new CSV file
df_concatenated.to_csv('specs_all_final.csv', index=False)


In [61]:
dfl=pd.read_csv('specs_all_final.csv')

In [ ]:
dfl

In [21]:
# dffinal = pd.read_csv('/Users/atharv/Desktop/Bikes_scrapping/notebooks/concatenated_file.csv')
# dffinal.drop_duplicates(inplace=True)
# duplicates = dffinal[dffinal.duplicated(subset=['Search Query', 'Specs Link'], keep=False)]
# print(duplicates)


# dffinal.to_csv('concatenated_fin.csv', index=False)

In [ ]:

# # dfz['Bike Name'].nunique()

# # print(dfz.isnull().sum())

# # dfz.to_csv('bike_specZ.csv', index=False)

In [ ]:
# ['Yamaha MT 15 V2', 'Yamaha R15 V4', 'Yamaha R15S',
#        'Yamaha FZS-FI V3', 'Yamaha RayZR 125 Fi Hybrid',
#        'Yamaha Aerox 155', 'Yamaha FZ-FI V3',
#        'Yamaha Fascino 125 Fi Hybrid', 'Yamaha FZS-FI V4', 'Yamaha FZ X',
#        'Yamaha R3', 'Yamaha MT-03', 'Yamaha FZ 25', 'Yamaha FZS 25'].sort

### Method for specs extraction only using specs page direct link

In [ ]:
#working fine for specz extraction using specs page link

# Initialize the browser
driver = webdriver.Chrome()

# Navigate to the URL
driver.get('https://www.bikedekho.com/ola-electric/s1-pro/specifications')


try:
    # Wait for the technical specifications section to be present
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'technicalSpecs')))

    # Find the technical specifications block
    tech_specs_block = driver.find_element(By.ID, 'technicalSpecs')

    # Find all h3 elements within the technical specifications block
    section_titles = tech_specs_block.find_elements(By.TAG_NAME, 'h3')

    for title_element in section_titles:
        section_title = title_element.text
        print(f"=== {section_title} ===")
        
        # Find the corresponding table for each section
        table = title_element.find_element(By.XPATH, './following-sibling::table[1]')
        
        # Find all rows in the table
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, 'td')
            if len(columns) == 2:
                feature = columns[0].text
                value = columns[1].text
                print(f"{feature}: {value}")
        
        print("\n")  # Separate sections with a newline

finally:
    # Close the browser
    driver.quit()


In [27]:
# df_yez_ult = pd.read_csv(".csv")

In [24]:
# df_yez_ult.drop_duplicates(inplace=True)

In [26]:
# df_yez_ult.to_csv('bike_spec_page_link_Yezdi_ultravioett.csv',index=False)

### Method for specs extraction like -- *Features*,*Price*,*Bike name*,*Sections* -- , using specs page direct link

In [37]:
#woking for data extraction form links final one 

# Read the CSV file containing the links
dflinks = pd.read_csv('bike_spec_page_link_Yezdi_ultravioett.csv')

# Initialize the browser
driver = webdriver.Chrome()

try:
    bike_details_list = []

    for link in dflinks['Specs Link']:
        # Navigate to the URL
        driver.get(link)

        # Wait for the bike name to be present
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h1')))

        # Extract bike name
        bike_name_full = driver.find_element(By.XPATH, '//h1').text
        bike_name = bike_name_full.split(' of ')[-1].strip()

        # Extract bike price
        try:
            bike_price_element = driver.find_element(By.XPATH, '//div[contains(@class, "price")]//i[contains(@class, "icon-cd_R")]//parent::span')
            
            if bike_price_element.is_displayed():
                bike_price_text = bike_price_element.text.split('Rs.')[-1].split('*')[0].strip()
                
                print(f"Bike Name: {bike_name}")
                print(f"Price: {bike_price_text}")
                
            else:
                print("Price element is not visible")
                bike_price_text = "N/A"

        except Exception as e:
            print(f"Error extracting bike price: {e}")
            bike_price_text = "N/A"

        bike_details = {
            'Bike Name': bike_name,
            'Price': bike_price_text,
            'Specs': []
        }

        # Wait for the technical specifications section to be present
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'technicalSpecs')))

        # Find the technical specifications block
        tech_specs_block = driver.find_element(By.ID, 'technicalSpecs')

        # Find all h3 elements within the technical specifications block
        section_titles = tech_specs_block.find_elements(By.TAG_NAME, 'h3')

        for title_element in section_titles:
            section_title = title_element.text

            # Find the corresponding table for each section
            table = title_element.find_element(By.XPATH, './following-sibling::table[1]')

            # Find all rows in the table
            rows = table.find_elements(By.TAG_NAME, 'tr')

            for row in rows:
                columns = row.find_elements(By.TAG_NAME, 'td')
                if len(columns) == 2:
                    feature = columns[0].text
                    value = columns[1].text

                    # Append the feature and value to the bike_details['Specs']
                    bike_details['Specs'].append({
                        'Feature': feature,
                        'Value': value,
                        'Section Title': section_title,
                    })

        # Append the bike_details to the bike_details_list
        bike_details_list.append(bike_details)

finally:
    # Close the browser
    driver.quit()

# Create a dataframe from the bike_details_list
df_bike_details = pd.DataFrame([
    {'Bike Name': bike['Bike Name'], 'Price': bike['Price'], **spec}
    for bike in bike_details_list
    for spec in bike['Specs']
])

# Print the dataframe
print(df_bike_details)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.79)
Stacktrace:
0   chromedriver                        0x00000001010b09c8 chromedriver + 5216712
1   chromedriver                        0x00000001010a8603 chromedriver + 5182979
2   chromedriver                        0x0000000100c1a527 chromedriver + 406823
3   chromedriver                        0x0000000100bf056e chromedriver + 234862
4   chromedriver                        0x0000000100c9102d chromedriver + 892973
5   chromedriver                        0x0000000100ca7038 chromedriver + 983096
6   chromedriver                        0x0000000100c884a3 chromedriver + 857251
7   chromedriver                        0x0000000100c58fe3 chromedriver + 663523
8   chromedriver                        0x0000000100c5992e chromedriver + 665902
9   chromedriver                        0x000000010106e8e0 chromedriver + 4946144
10  chromedriver    

<!-- df_bike_details.to_csv('bike_specall.csv', index=False) -->

In [46]:
# df_yezdi_ultra = pd.read_csv('bike_specs_yezdi_ultraviolet.csv')

# # Split the 'Feature' column into two columns based on '\n'
# df_yezdi_ultra[['Feature', 'Feature Info']] = df_yezdi_ultra['Feature'].str.split('\n', expand=True)

# df_yezdi_ultra.drop('Feature Info', axis=1, inplace=True)


# feature_info = df_yezdi_ultra['Feature Info']
# print(feature_info)


In [56]:
df_yezdi_ultra.to_csv("bike_specs_yezdi_ultraviolet.csv",index=False)

In [51]:
df_yezdi_ultra = pd.read_csv("bike_specs_yezdi_ultraviolett.csv")

In [54]:
df_yezdi_ultra.drop('Unnamed: 0', axis=1, inplace=True)


In [55]:
df_yezdi_ultra

,Bike Name,Price,Feature,Value,Section Title
0,Yezdi Roadster,2.06 - 2.13 Lakh,Engine Type,"Single cylinder, 4 Stroke, Liquid Cooled, DOHC",Engine and Transmission
1,Yezdi Roadster,2.06 - 2.13 Lakh,Displacement,334 cc,Engine and Transmission
2,Yezdi Roadster,2.06 - 2.13 Lakh,Max Torque,29.40 Nm @ 6500 rpm,Engine and Transmission
3,Yezdi Roadster,2.06 - 2.13 Lakh,No. of Cylinders,1,Engine and Transmission
4,Yezdi Roadster,2.06 - 2.13 Lakh,Cooling System,Liquid Cooled,Engine and Transmission
...,...,...,...,...,...
245,Ultraviolette F77,2.99 - 5.60 Lakh,Frame,Steel Trellis with Aluminium Bulk Head,Underpinnings
246,Ultraviolette F77,2.99 - 5.60 Lakh,Tubeless Tyre,Tubeless,Underpinnings
247,Ultraviolette F77,2.99 - 5.60 Lakh,Battery Warranty,"8 Years or 1,00,000 Km",What’s Included
248,Ultraviolette F77,2.99 - 5.60 Lakh,Vehicle Warranty,"3 Years or 30,000 Km",What’s Included


In [48]:
# feature_info.to_csv('feature_info.csv',index=False)

In [29]:
df_bike_details.to_csv('bike_specs_yezdi_ultraviolet.csv', index=False)

In [ ]:
# dffin = pd.read_csv('/Users/atharv/Desktop/Bikes_scrapping/notebooks/bike_specall.csv')
# duplicates = dffin[dffin.duplicated(subset=['Bike Name','Price','Feature','Value','Section Title'], keep=False)]
# print(duplicates)


In [ ]:
#working fine for bike main page data fetching 

def find_element_with_retry(driver, by, value, retries=3, delay=2):
    for _ in range(retries):
        try:
            element = WebDriverWait(driver, delay).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except (TimeoutException, StaleElementReferenceException) as e:
            print(f"Error finding element: {str(e)}. Retrying...")
            sleep(delay)
    raise Exception(f"Unable to find element {value} after {retries} retries.")

try:
    # Initialize the browser
    driver = webdriver.Chrome()

    # Navigate to the URL
    driver.get('https://www.bikedekho.com')

    # List of search queries
    search_queries = [
        'ola bike', 'Bajaj bike', 'TVS bike', 'Royal Enfield bike',
        'Honda bike', 'Yamaha bike', 'Suzuki bike', 'KTM bike',
        'Jawa bike', 'Benelli bike', 'Harley bike', 'Triumph bike',
        'Ducati bike', 'BMW bike', 'Kawasaki bike', 'Husqvarna bike',
        'Revolt bike', 'Ather bike','Aprillia bike'
    ]

    bike_details = []

    for search_query in search_queries:
        # Find the search text input element
        search_text_input = find_element_with_retry(driver, By.ID, 'cardekhosearchtext')

        # Input the search query into the search text input
        search_text_input.clear()
        search_text_input.send_keys(search_query)

        # Wait for a while to let the dropdown options appear
        sleep(2)

        # Re-find the dropdown options after waiting
        dropdown_option_xpath = '//*[@id="rf01"]/header/div[1]/div/div/div[2]/div/div/form/div/div/div/ul/li'
        dropdown_options = find_element_with_retry(driver, By.XPATH, dropdown_option_xpath, retries=5, delay=2)

        # Select the first option from the dropdown if available
        if dropdown_options:
            first_option = dropdown_options
            driver.execute_script("arguments[0].scrollIntoView();", first_option)
            first_option.click()

        sleep(2)
        
        bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')

        for index in range(len(bike_list_element.find_elements(By.TAG_NAME, 'li'))):
            try:
                # Re-find the bike list element to avoid stale element reference
                bike_list_element = find_element_with_retry(driver, By.CLASS_NAME, 'bikelist')
                
                item = bike_list_element.find_elements(By.TAG_NAME, 'li')[index]
                x_element = item.find_element(By.TAG_NAME, 'h3').find_element(By.TAG_NAME, 'a')
                link = x_element.get_attribute('href')
                bike_details.append(link)
                
                # Click on the link to navigate to the bike details page
                x_element.click()

                # Additional scraping from the new page (e.g., bike name, price, etc.)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//h1')))
                bike_name = driver.find_element(By.XPATH, '//h1').text
                bike_price = driver.find_element(By.XPATH, '//span[@class="price"]').text
                
                print(f"Bike Name: {bike_name}, Price: {bike_price}")

                # Navigate back to the previous page
                driver.execute_script("window.history.go(-1)")
                
                # Wait for the bike list to load again
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'bikelist')))
                
            except NoSuchElementException:
                print(f"No <h3> tag found for item {search_query}. Skipping...")
                continue
            except Exception as e:
                print(f"An error occurred while processing item {search_query}: {str(e)}")
                continue

    bikes_df_basicss = pd.DataFrame(bike_details, columns=['Bike URLs'])
    # print(bikes_df_basicss)

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    # Close the browser
    if driver:
        driver.quit()
